In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_reset, het_breuschpagan
from statsmodels.tsa.stattools import grangercausalitytests

# Verilerin yüklenmesi
data = pd.read_excel(r'C:\Users\ilayd\OneDrive\Masaüstü\Yeni klasör\veri.xlsx')

# Logaritmik dönüşüm
data_log = np.log(data[['kişi başı sağlık harcaması', 'co2 emisyonu', 'kişi başı gsyih']])

# Birim kök testi
def adf_test(series, title=''):
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    print(out)
    print('\n')

adf_test(data['kişi başı sağlık harcaması'], 'Kişi Başı Sağlık Harcaması (Logaritmik Dönüşüm Öncesi)')
adf_test(data['co2 emisyonu'], 'CO2 Emisyonu (Logaritmik Dönüşüm Öncesi)')
adf_test(data['kişi başı gsyih'], 'Kişi Başı GSYIH (Logaritmik Dönüşüm Öncesi)')

# Logaritmik dönüşüm sonrası birim kök testi
adf_test(data_log['kişi başı sağlık harcaması'], 'Kişi Başı Sağlık Harcaması (Logaritmik Dönüşüm Sonrası)')
adf_test(data_log['co2 emisyonu'], 'CO2 Emisyonu (Logaritmik Dönüşüm Sonrası)')
adf_test(data_log['kişi başı gsyih'], 'Kişi Başı GSYIH (Logaritmik Dönüşüm Sonrası)')

# OLS modeli oluşturma
model_before = sm.OLS(data['kişi başı sağlık harcaması'], sm.add_constant(data[['kişi başı gsyih']])).fit()

# Model spesifikasyon hata testleri
reset_test_before = linear_reset(model_before)
print("Ramsey RESET Testi (Doğrusal Olmama Testleri):")
print(reset_test_before)

_, p_value_before, _, _ = het_breuschpagan(model_before.resid, model_before.model.exog)
print("Heteroskedastisite Testi - Breusch-Pagan:")
print("p-value:", p_value_before)

# Engle-Granger nedensellik testi (logaritmik dönüşüm sonrası)
granger_test_log_after = grangercausalitytests(data_log[['kişi başı gsyih', 'kişi başı sağlık harcaması']], maxlag=2)
print("Engle-Granger Nedensellik Testi (Logaritmik Dönüşüm Sonrası):")
print(granger_test_log_after)

# Eşbütünleşme testi (logaritmik dönüşüm sonrası)
coint_result_log_after = coint_johansen(data_log[['kişi başı sağlık harcaması', 'kişi başı gsyih']], det_order=0, k_ar_diff=1)
print('Johansen Testi Sonuçları (Logaritmik Dönüşüm Sonrası):')
print(coint_result_log_after.lr1)  # trace statistic
print(coint_result_log_after.cvt)  # critical value trace statistic
print(coint_result_log_after.lr2)  # max eigen statistic
print(coint_result_log_after.cvm)  # critical value max eigen statistic

# KSS testi (logaritmik dönüşüm sonrası)
def kss_test(series):
    kss_statistic = series.diff().kurt()
    print(f'Kurtosis-Based Stability Test: {series.name}')
    print(f'KSS Statistic: {kss_statistic}')
    print(f'Series Name: {series.name}')
    print('\n')

kss_test(data_log['kişi başı sağlık harcaması'])
kss_test(data_log['co2 emisyonu'])
kss_test(data_log['kişi başı gsyih'])

# Grafiklerin oluşturulması (logaritmik dönüşüm sonrası)
plt.figure(figsize=(10,6))
plt.plot(data['yıllar'], data_log['kişi başı sağlık harcaması'], label='Log Kişi Başı Sağlık Harcaması')
plt.plot(data['yıllar'], data_log['co2 emisyonu'], label='Log CO2 Emisyonu')
plt.plot(data['yıllar'], data_log['kişi başı gsyih'], label='Log Kişi Başı GSYIH')
plt.xlabel('Yıllar')
plt.ylabel('Logaritmik Değerler')
plt.legend()
plt.title('Logaritmik Dönüşüm Sonrası Değerler')
plt.show()

# Sonuçların tablo olarak çıkarılması
data_log.to_excel(r'C:\Users\ilayd\OneDrive\Masaüstü\Yeni klasör\log_transform_data.xlsx', index=False)

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_reset, het_breuschpagan
from statsmodels.tsa.stattools import grangercausalitytests

# Verilerin yüklenmesi
data = pd.read_excel(r'C:\Users\ilayd\OneDrive\Masaüstü\Yeni klasör\veri.xlsx')

# OLS modeli oluşturma
model_before = sm.OLS(data['kişi başı sağlık harcaması'], sm.add_constant(data[['kişi başı gsyih']])).fit()

# Model spesifikasyon hata testleri (öncesi)
reset_test_before = linear_reset(model_before)
breusch_pagan_before = het_breuschpagan(model_before.resid, model_before.model.exog)

# Heteroskedastisite Testi - Breusch-Pagan (öncesi)
bp_pvalue_before = breusch_pagan_before[1]

# Birim kök testi (logaritmik dönüşüm öncesi)
def adf_test(series):
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    return out

adf_results_before = {
    'Kişi Başı Sağlık Harcaması': adf_test(data['kişi başı sağlık harcaması']),
    'CO2 Emisyonu': adf_test(data['co2 emisyonu']),
    'Kişi Başı GSYIH': adf_test(data['kişi başı gsyih'])
}

# Logaritmik dönüşüm
data_log = np.log(data[['kişi başı sağlık harcaması', 'co2 emisyonu', 'kişi başı gsyih']])

# OLS modeli oluşturma (sonrası)
model_after = sm.OLS(data_log['kişi başı sağlık harcaması'], sm.add_constant(data_log[['kişi başı gsyih']])).fit()

# Model spesifikasyon hata testleri (sonrası)
reset_test_after = linear_reset(model_after)
breusch_pagan_after = het_breuschpagan(model_after.resid, model_after.model.exog)

# Heteroskedastisite Testi - Breusch-Pagan (sonrası)
bp_pvalue_after = breusch_pagan_after[1]

# Birim kök testi (logaritmik dönüşüm sonrası)
def adf_test_log(series):
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    return out

adf_results_after = {
    'Kişi Başı Sağlık Harcaması': adf_test_log(data_log['kişi başı sağlık harcaması']),
    'CO2 Emisyonu': adf_test_log(data_log['co2 emisyonu']),
    'Kişi Başı GSYIH': adf_test_log(data_log['kişi başı gsyih'])
}

# KSS testi (öncesi)
def kss_test(series):
    kss_statistic = series.diff().kurt()
    return pd.Series({
        'KSS Statistic': kss_statistic
    })

kss_results_before = {
    'Kişi Başı Sağlık Harcaması': kss_test(data['kişi başı sağlık harcaması']),
    'CO2 Emisyonu': kss_test(data['co2 emisyonu']),
    'Kişi Başı GSYIH': kss_test(data['kişi başı gsyih'])
}

# KSS testi (logaritmik dönüşüm sonrası)
def kss_test_log(series):
    kss_statistic = series.diff().kurt()
    return pd.Series({
        'KSS Statistic': kss_statistic
    })

kss_results_after = {
    'Kişi Başı Sağlık Harcaması': kss_test_log(data_log['kişi başı sağlık harcaması']),
    'CO2 Emisyonu': kss_test_log(data_log['co2 emisyonu']),
    'Kişi Başı GSYIH': kss_test_log(data_log['kişi başı gsyih'])
}

# Sonuçların tablo olarak çıkarılması (Excel)
with pd.ExcelWriter(r'C:\Users\ilayd\OneDrive\Masaüstü\Yeni klasör\analiz_sonuclari.xlsx') as writer:
    pd.DataFrame.from_dict(adf_results_before, orient='index').to_excel(writer, sheet_name='ADF Test (Öncesi)')
    pd.DataFrame.from_dict(adf_results_after, orient='index').to_excel(writer, sheet_name='ADF Test (Sonrası)')
    pd.DataFrame.from_dict(kss_results_before, orient='index').to_excel(writer, sheet_name='KSS Test (Öncesi)')
    pd.DataFrame.from_dict(kss_results_after, orient='index').to_excel(writer, sheet_name='KSS Test (Sonrası)')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_reset, het_breuschpagan
from statsmodels.tsa.stattools import grangercausalitytests

# Verilerin yüklenmesi
data = pd.read_excel(r'C:\Users\ilayd\OneDrive\Masaüstü\Yeni klasör\veri.xlsx')

# OLS modeli oluşturma
model_before = sm.OLS(data['kişi başı sağlık harcaması'], sm.add_constant(data[['kişi başı gsyih']])).fit()

# Model spesifikasyon hata testleri
reset_test_before = linear_reset(model_before)
print("Ramsey RESET Testi (Doğrusal Olmama Testleri):")
print(reset_test_before)

_, p_value_before, _, _ = het_breuschpagan(model_before.resid, model_before.model.exog)
print("Heteroskedastisite Testi - Breusch-Pagan:")
print("p-value:", p_value_before)

# Birim kök testi (logaritmik dönüşüm öncesi)
def adf_test(series, title=''):
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out['Critical Value (%s)'%key] = value
    print(out)
    print('\n')
    return out

adf_test_before_health = adf_test(data['kişi başı sağlık harcaması'], 'Kişi Başı Sağlık Harcaması Öncesi')
adf_test_before_co2 = adf_test(data['co2 emisyonu'], 'CO2 Emisyonu Öncesi')
adf_test_before_gsyih = adf_test(data['kişi başı gsyih'], 'Kişi Başı GSYIH Öncesi')

# Logaritmik dönüşüm
data_log = np.log(data[['kişi başı sağlık harcaması', 'co2 emisyonu', 'kişi başı gsyih']])

# OLS modeli oluşturma (sonrası)
model_after = sm.OLS(data_log['kişi başı sağlık harcaması'], sm.add_constant(data_log[['kişi başı gsyih']])).fit()

# Model spesifikasyon hata testleri (sonrası)
reset_test_after = linear_reset(model_after)
print("Ramsey RESET Testi (Doğrusal Olmama Testleri - Sonrası):")
print(reset_test_after)

_, p_value_after, _, _ = het_breuschpagan(model_after.resid, model_after.model.exog)
print("Heteroskedastisite Testi - Breusch-Pagan (Sonrası):")
print("p-value:", p_value_after)

# Birim kök testi (logaritmik dönüşüm sonrası)
adf_test_after_health = adf_test(data_log['kişi başı sağlık harcaması'], 'Kişi Başı Sağlık Harcaması Sonrası')
adf_test_after_co2 = adf_test(data_log['co2 emisyonu'], 'CO2 Emisyonu Sonrası')
adf_test_after_gsyih = adf_test(data_log['kişi başı gsyih'], 'Kişi Başı GSYIH Sonrası')

# Excel'e yazdırma
with pd.ExcelWriter(r'C:\Users\ilayd\OneDrive\Masaüstü\Yeni klasör\birim_kok_testleri.xlsx') as writer:
    adf_test_before_health.to_excel(writer, sheet_name='Öncesi - Sağlık Harcaması')
    adf_test_before_co2.to_excel(writer, sheet_name='Öncesi - CO2 Emisyonu')
    adf_test_before_gsyih.to_excel(writer, sheet_name='Öncesi - GSYIH')
    adf_test_after_health.to_excel(writer, sheet_name='Sonrası - Sağlık Harcaması')
    adf_test_after_co2.to_excel(writer, sheet_name='Sonrası - CO2 Emisyonu')
    adf_test_after_gsyih.to_excel(writer, sheet_name='Sonrası - GSYIH')

# Dosyanın kaydedildiği yolu ekrana yazdırma
print("Birim kök testleri Excel dosyası burada kaydedildi: C:\\Users\\ilayd\\OneDrive\\Masaüstü\\Yeni klasör\\birim_kok_testleri.xlsx")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

# Verilerin yüklenmesi
data = pd.read_excel(r'C:\Users\ilayd\OneDrive\Masaüstü\Yeni klasör\veri.xlsx')

# ADF testi fonksiyonu
def adf_test(series, title='', ax=None):
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    
    # Grafik oluşturma
    if ax is None:
        fig, ax = plt.subplots()
    
    ax.plot(series.index, series.values, label='Data')
    ax.set_title(f'{title} - ADF Test')
    ax.axhline(y=out['Critical Value (1%)'], color='r', linestyle='--', label='1% Critical Value')
    ax.axhline(y=out['Critical Value (5%)'], color='g', linestyle='--', label='5% Critical Value')
    ax.axhline(y=out['Critical Value (10%)'], color='b', linestyle='--', label='10% Critical Value')
    ax.legend()
    
    return out, ax

# Kişi başı sağlık harcaması için ADF testi
fig, ax1 = plt.subplots()
adf_results_health_before, ax1 = adf_test(data['kişi başı sağlık harcaması'], title='Kişi Başına Sağlık Harcaması logÖncesi', ax=ax1)

fig, ax2 = plt.subplots()
adf_results_health_after, ax2 = adf_test(np.log(data['kişi başı sağlık harcaması']), title='Kişi Başına Sağlık Harcaması logSonrası', ax=ax2)

# CO2 Emisyonu için ADF testi
fig, ax3 = plt.subplots()
adf_results_co2_before, ax3 = adf_test(data['co2 emisyonu'], title='CO2 Emisyonu logÖncesi', ax=ax3)

fig, ax4 = plt.subplots()
adf_results_co2_after, ax4 = adf_test(np.log(data['co2 emisyonu']), title='CO2 Emisyonu logSonrası', ax=ax4)

# Kişi başı GSYIH için ADF testi
fig, ax5 = plt.subplots()
adf_results_gsyih_before, ax5 = adf_test(data['kişi başı gsyih'], title='Kişi Başına GSYIH logÖncesi', ax=ax5)

fig, ax6 = plt.subplots()
adf_results_gsyih_after, ax6 = adf_test(np.log(data['kişi başı gsyih']), title='Kişi Başına GSYIH logSonrası', ax=ax6)

plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Verilerin yüklenmesi
data = pd.read_excel(r'C:\Users\ilayd\OneDrive\Masaüstü\Yeni klasör\veri.xlsx')

# KSS testi fonksiyonu
def kss_test(series, title='', ax=None):
    kss_statistic = series.diff().kurt()
    
    # Grafik oluşturma
    if ax is None:
        fig, ax = plt.subplots()
    
    ax.plot(series.index, series.values, label='Data')
    ax.set_title(f'{title} - KSS Test')
    ax.axhline(y=kss_statistic, color='r', linestyle='--', label='KSS Statistic')
    ax.legend()
    
    return kss_statistic, ax

# Kişi başı sağlık harcaması için KSS testi
fig, ax1 = plt.subplots()
kss_stat_health_before, ax1 = kss_test(data['kişi başı sağlık harcaması'], title='Kişi Başına Sağlık Harcaması log Öncesi', ax=ax1)

fig, ax2 = plt.subplots()
kss_stat_health_after, ax2 = kss_test(np.log(data['kişi başı sağlık harcaması']), title='Kişi Başına Sağlık Harcaması log Sonrası', ax=ax2)

# CO2 Emisyonu için KSS testi
fig, ax3 = plt.subplots()
kss_stat_co2_before, ax3 = kss_test(data['co2 emisyonu'], title='CO2 Emisyonu log Öncesi', ax=ax3)

fig, ax4 = plt.subplots()
kss_stat_co2_after, ax4 = kss_test(np.log(data['co2 emisyonu']), title='CO2 Emisyonu log Sonrası', ax=ax4)

# Kişi başı GSYIH için KSS testi
fig, ax5 = plt.subplots()
kss_stat_gsyih_before, ax5 = kss_test(data['kişi başı gsyih'], title='Kişi Başına GSYIH log Öncesi', ax=ax5)

fig, ax6 = plt.subplots()
kss_stat_gsyih_after, ax6 = kss_test(np.log(data['kişi başı gsyih']), title='Kişi Başına GSYIH log Sonrası', ax=ax6)

plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Verilerin yüklenmesi
data = pd.read_excel(r'C:\Users\ilayd\OneDrive\Masaüstü\Yeni klasör\veri.xlsx')

# Logaritmik dönüşüm
data_log = np.log(data[['kişi başı sağlık harcaması', 'co2 emisyonu', 'kişi başı gsyih']])

# Kişi başı GSYİH ile kişi başı sağlık harcaması arasında doğrusal olmayan model oluşturma ve kalıntıları hesaplama
x1 = data_log['kişi başı gsyih']
y1 = data_log['kişi başı sağlık harcaması']
coeffs1 = np.polyfit(np.log(x1), np.log(y1), 1)
y_pred1 = np.exp(coeffs1[1]) * x1 ** coeffs1[0]
residuals1 = y1 - y_pred1

# Kişi başı GSYİH ve CO2 emisyonu arasında doğrusal olmayan model oluşturma ve kalıntıları hesaplama
x2 = data_log['kişi başı gsyih']
y2 = data_log['co2 emisyonu']
coeffs2 = np.polyfit(np.log(x2), np.log(y2), 1)
y_pred2 = np.exp(coeffs2[1]) * x2 ** coeffs2[0]
residuals2 = y2 - y_pred2

# Augmented Dickey-Fuller (ADF) Birim Kök Testi
adf_test1 = sm.tsa.stattools.adfuller(residuals1)
print(f'ADF Test Statistic (kişi başı gsyih ve sağlık harcamaları): {adf_test1[0]}')
print(f'p-value (kişi başı gsyih ve sağlık harcamaları): {adf_test1[1]}')

adf_test2 = sm.tsa.stattools.adfuller(residuals2)
print(f'ADF Test Statistic (kişi başı gsyih ve CO2 emisyonu): {adf_test2[0]}')
print(f'p-value (kişi başı gsyih ve CO2 emisyonu): {adf_test2[1]}')

# Doğrusal Olmayan KSS Eşbütünleşme Testi (Nonlinear Cointegration Test)
coint_test1 = sm.tsa.stattools.coint(x1, residuals1)
print(f'Cointegration test statistic (kişi başı gsyih ve sağlık harcamaları): {coint_test1[0]}')
print(f'p-value (kişi başı gsyih ve sağlık harcamaları): {coint_test1[1]}')

coint_test2 = sm.tsa.stattools.coint(x2, residuals2)
print(f'Cointegration test statistic (kişi başı gsyih ve CO2 emisyonu): {coint_test2[0]}')
print(f'p-value (kişi başı gsyih ve CO2 emisyonu): {coint_test2[1]}')

# Kalanların grafiği
plt.figure(figsize=(14, 8))

# Kişi başı gsyih ve sağlık harcamaları için KSS istatistiği grafiği
plt.subplot(2, 1, 1)
plt.plot(data['yıllar'], residuals1, label='Residuals (kişi başı gsyih ve sağlık harcamaları)')
plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
plt.xlabel('Yıllar')
plt.ylabel('Kalanlar')
plt.legend()
plt.title('Doğrusal Olmayan KSS Eşbütünleşme Testi Kalanları (kişi başı gsyih ve sağlık harcamaları)')

# Kişi başı gsyih ve CO2 emisyonu için KSS istatistiği grafiği
plt.subplot(2, 1, 2)
plt.plot(data['yıllar'], residuals2, label='Residuals (kişi başı gsyih ve CO2 emisyonu)')
plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
plt.xlabel('Yıllar')
plt.ylabel('Kalanlar')
plt.legend()
plt.title('Doğrusal Olmayan KSS Eşbütünleşme Testi Kalanları (kişi başı gsyih ve CO2 emisyonu)')

plt.tight_layout()
plt.show()